In [1]:
from keras.applications.vgg16 import VGG16

# 既存の 1,000 クラス分類を行わず、 include_top=False で出力層を含まないモデルとする
vgg16 = VGG16(include_top=False, input_shape=(224, 224, 3))

vgg16.summary()

/Users/jDB/.pyenv/versions/3.6.5/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [2]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten


def build_transfer_model(vgg16):
    # VGG16 のモデルから新たなモデル生成
    model = Sequential(vgg16.layers)

    # 指定範囲は重みの再学習を行わないよう設定
    for layer in model.layers[:15]:
        layer.trainable = False

    # 出力層の追加
    model.add(Flatten())
    # 出力サイズ 256
    model.add(Dense(256, activation="relu"))
    model.add(Dropout(0.5))
    # 出力サイズ 1 (二値分類)
    model.add(Dense(1, activation="relu"))

    return model


model = build_transfer_model(vgg16)

In [3]:
from keras.optimizers import SGD

model.compile(
    loss="binary_crossentropy",
    # 既存の重みを活かすため学習率を低めに設定
    optimizer=SGD(lr=1e-4, momentum=0.9),
    metrics=["accuracy"]
)

In [4]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [5]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import preprocess_input


# 学習用画像をロードするためのジェネレータ
image_data_generate_train = ImageDataGenerator(
    # スケール変換
    rescale=1/255.,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    preprocessing_function=preprocess_input
)

image_data_generate_train

In [7]:
# ジェネレートした画像をロードするイテレータ生成

IMG_ROOT_PATH = "/Applications/MAMP/htdocs/lessons/python/tensorflow/dataset/img"
# 学習データ
img_iter_train = image_data_generate_train.flow_from_directory(
    # 画像パス
    IMG_ROOT_PATH + "/shrine_temple/train",
    # 画像サイズ
    target_size=(224, 224),
    # 学習時の画像バッチ数
    batch_size=16,
    # 二値分類
    class_mode="binary"
)

# 検証データ
img_iter_test = image_data_generate_train.flow_from_directory(
    IMG_ROOT_PATH + "/shrine_temple/validation",
    target_size=(224, 224),
    batch_size=16,
    class_mode="binary"
)

Found 600 images belonging to 2 classes.
Found 200 images belonging to 2 classes.


In [8]:
from datetime import datetime
import os


# モデル保存用のディレクトリ
model_dir = os.path.join(
    "models",
    datetime.now().strftime("%y%m%d_%H%M")
)
os.makedirs(model_dir, exist_ok=True)

model_dir

'models/180726_2301'

In [9]:
dir_weights = os.path.join(model_dir, "weights")
os.makedirs(dir_weights, exist_ok=True)

dir_weights

'models/180726_2301/weights'

In [11]:
import json
import pickle


# ネットワークの保存
model_json = os.path.join(model_dir, "model.json")
with open(model_json, "w") as f:
    json.dump(model.to_json(), f)

# 学習時の正解ラベルの保存
model_classes = os.path.join(model_dir, "classes.pkl")
with open(model_classes, "wb") as f:
    pickle.dump(img_iter_train.class_indices, f)

In [19]:
import math


# バッチサイズから 1エポックに必要なバッチ数を計算
batch_size = 16
steps_per_epoch = math.ceil(
    img_iter_train.samples / batch_size
)
validation_steps = math.ceil(
    img_iter_test.samples / batch_size
)

In [20]:
# 1エポックに必要なステップ数
steps_per_epoch

38

In [21]:
validation_steps

13

In [15]:
from keras.callbacks import ModelCheckpoint, CSVLogger


# エポック単位でモデルの重みや損失の値をファイルに保存
checkpoint_filepath = os.path.join(dir_weights, 'ep_{epoch:02d}_ls_{loss:.1f}.h5')
checkpoint = ModelCheckpoint(
    checkpoint_filepath,
    monitor="loss",
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode="auto",
    period=5
)

csv_filepath = os.path.join(model_dir, "loss.csv")
csv = CSVLogger(csv_filepath, append=True)

In [16]:
checkpoint_filepath

'models/180726_2301/weights/ep_{epoch:02d}_ls_{loss:.1f}.h5'

In [17]:
csv_filepath

'models/180726_2301/loss.csv'

In [18]:
csv

In [22]:
n_epoch = 30

# 学習
history = model.fit_generator(
    img_iter_train,
    steps_per_epoch,
    # エポック数
    epochs=n_epoch,
    validation_data=img_iter_test,
    validation_steps=validation_steps,
    callbacks=[checkpoint, csv]
)

Epoch 1/10
38/38 [==============================] - 530s 14s/step - loss: 7.8512 - acc: 0.0428 - val_loss: 7.9712 - val_acc: 0.0000e+00
Epoch 2/10
38/38 [==============================] - 806s 21s/step - loss: 7.9456 - acc: 0.0000e+00 - val_loss: 7.9712 - val_acc: 0.0000e+00
Epoch 3/10
 9/38 [======>.......................] - ETA: 3:54 - loss: 8.4140 - acc: 0.0000e+00

KeyboardInterrupt: 

In [ ]:
from utils import load_random_imgs


# ランダムに評価用画像を抽出
test_data_dir = IMG_ROOT_PATH + "/shrine_temple/test/unknown"
x_test, true_labels = load_random_imgs(
    test_data_dir,
    seed=1
)
x_test

In [ ]:
x_test_preproc = preprocess_input(x_test.copy()) / 255.
# 画像が「寺社」である確率
probs = model.predict(x_test_preproc)

probs

In [ ]:
from utils import show_test_samples


# 評価用画像の表示
show_test_samples(
    x_test,
    probs,
    img_iter_train.class_indices,
    true_labels
)